## Database Structure Design



| Relationship                          | Type  | Explanation                                                                 |
| ------------------------------------- | ----- | --------------------------------------------------------------------------- |
| `employee` → `employee_demographics`  | **1:1** | Each employee has a single row with their demographic information           |
| `employee` → `employee_professional`  | **1:1** | Each employee has a single registered professional profile                  |
| `employee` → `employee_financial`     | **1:1** | Each employee has a unique set of financial data                            |
| `employee` → `employee_satisfaction`  | **1:1** | Each employee has a single satisfaction and performance evaluation          |


## Tables and Columns Structure



### employee
- employeenumber (PK)
- attrition

### employee_demographics
- employeenumber (FK)
- age
- gender
- marital_status
- date_birth
- generation
- education
- educationfield
- distance_from_home

### employee_professional
- employeenumber (FK)
- job_role
- department
- years_at_company
- num_companies_worked
- over_time
- training_times_last_year
- years_in_current_role
- job_level
- business_travel
- standard_hours
- remotework

### employee_financial
- employeenumber (FK)
- monthly_income
- monthly_rate
- salary
- stock_option_level
- daily_rate
- percent_salary_hike

### employee_satisfaction
- employeenumber (FK)
- job_satisfaction
- performance_rating
- relationship_satisfaction
- work_life_balance
- environment_satisfaction
- job_involvement


In [18]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd

### Create a Database in MySQL Workbench Using MySQL Connector


In [19]:
try:
  cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='bluepeak_technologies')
  print("ok")

except mysql.connector.Error as err:
# If there is an error with the password, return an "Access denied" message, since we are having issues with the password.
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
# If the error is not related to the password, it may be because the database does not exist return a message saying that the database does not exist.
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

ok


In [20]:
cursor = cnx.cursor()

In [21]:
import pandas as pd

df = pd.read_csv('/Users/ananobre/Adalab_Bootcamp/Projetos Pessoais para Git/Talent-Optimization-for-Corporation-HR-Data-Analytics-1/Data-Analytics-and-Vizualization/hr_data_cleaned.csv')
df.head(1)


,Unnamed: 0,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,...,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework,generation
0,0,53,No,NaN,2015.722222,research & development,6,3,NaN,1,...,5,3.0,20,15.0,15,15,1972,195370.0,True,Gen X


### employee
- employeenumber (PK)
- attrition

In [22]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS employee (
    employeenumber INT PRIMARY KEY,
    attrition VARCHAR(10)
);
""")


In [23]:
employee_data = df[['employeenumber', 'attrition']].values.tolist()
cursor.executemany("""
    INSERT INTO employee (employeenumber, attrition)
    VALUES (%s, %s)
""", employee_data)

# The `.tolist()` method is used to convert a pandas object 
# (such as a Series or a DataFrame) into a Python list.

# When applied to a Series, it returns a simple list:
#     [value1, value2, value3, ...]

# When applied to a DataFrame (as in our case),
#     it returns a list of lists:
#     [[row1_col1, row1_col2, ...], [row2_col1, row2_col2, ...], ...]

# This is especially useful when using `cursor.executemany()` in MySQL,
# which requires a list of tuples or lists as input to insert multiple records.





### employee_demographics
- employeenumber (FK)
- age
- gender
- marital_status
- date_birth
- generation
- education
- educationfield
- distance_from_home


In [24]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS employee_demographics (
    employeenumber INT PRIMARY KEY,
    age INT,
    gender VARCHAR(20),
    marital_status VARCHAR(20),
    date_birth INT,
    generation VARCHAR(20),
    education INT,
    educationfield VARCHAR(100),
    distance_from_home INT,
    FOREIGN KEY (employeenumber) REFERENCES employee(employeenumber)
);
""")


In [25]:
demo_data = df[['employeenumber', 'age', 'gender', 'maritalstatus', 'datebirth','generation', 'education', 'educationfield', 
                'distancefromhome']].rename(columns={'maritalstatus': 'marital_status'}).where(pd.notnull(df), None).values.tolist()

cursor.executemany("""
    INSERT INTO employee_demographics (
        employeenumber, age, gender, marital_status, date_birth,
        generation, education, educationfield, distance_from_home
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""", demo_data)

cnx.commit()
print(" Dados inseridos con exito en la tabla employee_demographics.")

# .where(pd.notnull(df) is used to replace all missing values (NaN)
# in a DataFrame with Python's null value (`None`).

# Why is this necessary?
# - MySQL does not recognize `NaN` (Not a Number) as a null value.
# - When inserting data from pandas using `executemany()`, 
#   null values must be represented as `None`, which is accepted by MySQL.


 Dados inseridos con exito en la tabla employee_demographics.



### employee_professional
- employeenumber (FK)
- job_role
- department
- years_at_company
- num_companies_worked
- over_time
- training_times_last_year
- years_in_current_role
- job_level
- business_travel
- standard_hours
- remotework


In [26]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS employee_professional (
    employeenumber INT PRIMARY KEY,
    job_role VARCHAR(100),
    department VARCHAR(100),
    years_at_company INT,
    num_companies_worked INT,
    over_time VARCHAR(10),
    training_times_last_year INT,
    years_in_current_role FLOAT,
    job_level INT,
    business_travel VARCHAR(50),
    standard_hours VARCHAR(10),
    remotework BOOLEAN,
    FOREIGN KEY (employeenumber) REFERENCES employee(employeenumber)
);
""")


In [27]:
prof_df = df[['employeenumber', 'jobrole', 'department', 'yearsatcompany',
              'numcompaniesworked', 'overtime', 'trainingtimeslastyear',
              'yearsincurrentrole', 'joblevel', 'businesstravel',
              'standardhours', 'remotework']].rename(columns={
    'yearsatcompany': 'years_at_company',
    'numcompaniesworked': 'num_companies_worked',
    'overtime': 'over_time',
    'trainingtimeslastyear': 'training_times_last_year',
    'yearsincurrentrole': 'years_in_current_role',
    'joblevel': 'job_level',
    'businesstravel': 'business_travel',
    'standardhours': 'standard_hours'
})

# Replace NaN with None in the reduced DataFrame
prof_df = prof_df.where(pd.notnull(prof_df), None)
# Convert to list
prof_data = prof_df.values.tolist()
# Insert into the database
cursor.executemany("""
    INSERT INTO employee_professional (
        employeenumber, job_role, department, years_at_company,
        num_companies_worked, over_time, training_times_last_year,
        years_in_current_role, job_level, business_travel,
        standard_hours, remotework
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", prof_data)


print("Dados inseridos com sucesso na tabela employee_professional.")


Dados inseridos com sucesso na tabela employee_professional.



### employee_financial
- employeenumber (FK)
- monthly_income
- monthly_rate
- salary
- stock_option_level
- daily_rate
- percent_salary_hike


In [28]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS employee_financial (
    employeenumber INT PRIMARY KEY,
    monthly_income FLOAT,
    monthly_rate FLOAT,
    salary FLOAT,
    stock_option_level INT,
    daily_rate FLOAT,
    percent_salary_hike INT,
    FOREIGN KEY (employeenumber) REFERENCES employee(employeenumber)
);
""")


In [29]:
fin_data = df[['employeenumber', 'monthlyincome_$', 'monthlyrate_$', 'salary',
               'stockoptionlevel', 'dailyrate', 'percentsalaryhike']] \
               .rename(columns={
                   'monthlyincome_$': 'monthly_income',
                   'monthlyrate_$': 'monthly_rate',
                   'stockoptionlevel': 'stock_option_level',
                   'dailyrate': 'daily_rate',
                   'percentsalaryhike': 'percent_salary_hike'
               }).values.tolist()

cursor.executemany("""
    INSERT INTO employee_financial (
        employeenumber, monthly_income, monthly_rate, salary,
        stock_option_level, daily_rate, percent_salary_hike
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s)
""", fin_data)



### employee_satisfaction
- employeenumber (FK)
- job_satisfaction
- performance_rating
- relationship_satisfaction
- work_life_balance
- environment_satisfaction
- job_involvement


In [30]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS employee_satisfaction (
    employeenumber INT PRIMARY KEY,
    job_satisfaction INT,
    performance_rating FLOAT,
    relationship_satisfaction INT,
    work_life_balance VARCHAR(50),
    environment_satisfaction INT,
    job_involvement INT,
    FOREIGN KEY (employeenumber) REFERENCES employee(employeenumber)
);
""")


In [31]:
sat_data = df[['employeenumber', 'jobsatisfaction', 'performancerating',
               'relationshipsatisfaction', 'worklifebalance',
               'environmentsatisfaction', 'jobinvolvement']] \
               .rename(columns={
                   'jobsatisfaction': 'job_satisfaction',
                   'performancerating': 'performance_rating',
                   'relationshipsatisfaction': 'relationship_satisfaction',
                   'worklifebalance': 'work_life_balance',
                   'environmentsatisfaction': 'environment_satisfaction',
                   'jobinvolvement': 'job_involvement'
               }).values.tolist()

cursor.executemany("""
    INSERT INTO employee_satisfaction (
        employeenumber, job_satisfaction, performance_rating,
        relationship_satisfaction, work_life_balance,
        environment_satisfaction, job_involvement
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s)
""", sat_data)


In [32]:
cnx.commit()
print("Dados inseridos com sucesso em todas as tabelas.")


Dados inseridos com sucesso em todas as tabelas.


In [33]:
config = {
  'user': 'root',
  'password': 'AlumnaAdalab',
  'host': '127.0.0.1',
  'database': 'tienda',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

cnx.close()